# Bag of Visual words (or Features) of SIFT descriptors for SVM classification for detection of cars in a carpark

In [1]:
# all required modules
import cv2
from skimage.transform import pyramid_gaussian
from sklearn import svm
from sklearn.cluster import KMeans
from scipy.spatial import distance
import numpy as np
from matplotlib import pyplot as plt
import glob
import time
import os

## Functions

In [2]:
# Import training images and put into a dictionary
def training_images_to_dictionary(file_path):
    image_dictionary = {}
    for folders in os.listdir(file_path):
        path = glob.glob(file_path + folders + "/*.jpg")
        catergory = []
        for img in path:
            img = cv2.imread(img, 0)
            #img = cv2.resize(img,(256,256))
            catergory.append(img)
            image_dictionary[folders] = catergory
    return(image_dictionary)

In [3]:
# Creates an instance of SIFT
def init_sift():
    sift = cv2.xfeatures2d.SIFT_create()
    return(sift)

In [4]:
# Computes SIFT keypoints and Descriptors, returns descriptors only
def create_features(image, feature_algorithm):
    kp, des = feature_algorithm.detectAndCompute(image, None)
    return(des)

In [5]:
# Returns a dictionary of features sorted into class
def extract_features_and_order(images):
    dictionary = {}
    for key, value in images.items():
        features = []
        for img in value:
            features.append(create_features(img, sift))
        dictionary[key] = features
    return(dictionary)

In [6]:
# Returns features (descriptors) in an unordered list
def extract_features(images):
    feature_list = []
    for key, value in images.items():
        for img in value:
            feature_list.extend(create_features(img, sift))
    return(feature_list)

In [7]:
# Return features clustered
def cluster_algorithm(features, num_of_clusters):
    kmeans = KMeans(n_clusters = num_of_clusters)
    kmeans.fit(features)
    cluster = kmeans.cluster_centers_ 
    return(cluster)

In [8]:
# Creates histogram bins
def find_index(image, center):
    count = 0
    ind = 0
    for i in range(len(center)):
        if(i == 0):
            count = distance.euclidean(image, center[i]) 
           #count = L1_dist(image, center[i])
        else:
            dist = distance.euclidean(image, center[i]) 
            #dist = L1_dist(image, center[i])
            if(dist < count):
                ind = i
                count = dist
    return(ind)

In [9]:
# creates histogram or bag of visual words (Bag of Features)
def image_class(all_bovw, centers):
    dict_feature = {}
    for key,value in all_bovw.items():
        category = []
        for img in value:
            histogram = np.zeros(len(centers))
            for each_feature in img:
                ind = find_index(each_feature, centers)
                histogram[ind] += 1
            category.append(histogram)
        dict_feature[key] = category
    return(dict_feature)

In [10]:
# creates a histogram for a window
def histogram(image, centres):
    histogram = np.zeros(len(centres))
    for each_feature in image:
        ind = find_index(each_feature, centres)
        histogram[ind] += 1
    return(histogram)

In [11]:
# function to declare and define the Support Vector Machine (SVM)
def SVM_parameters(gam, c, Prob):
    clf = svm.SVC(gamma = gam, C = c, probability = Prob)
    return(clf)

In [12]:
# function to train the SVM with data(X), and labels (Y)
def SVM_train(clf,X,Y):
    clf.fit(X,Y)

In [13]:
# sliding window fuction to evaluate the test image in sections
def sliding_window(image, stepSize, windowSize):
    # slide a window across the image
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            # yield the current window
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])      

In [24]:
def nms(bound_boxes, overlap_threshold):
    valid_boxes = []
    # Convert to np array
    bound_boxes = np.array(bound_boxes, ndmin = 2)
    # Sort array by first coloum (probability) in decending order
    bound_boxes = sorted(bound_boxes, key=lambda x:x[0], reverse = True)
    while(len(bound_boxes) > 0):    
        # add box coordinates with greatest probabilty to return list
        #valid_boxes.insert(0, (bound_boxes[0][1:].astype("int")))
        valid_boxes.insert(0, (bound_boxes[0].astype("int")))
        # remove from bound box list
        bound_boxes.pop(0)
        
        # array index
        index = 0
        
        # compare selected box with all others
        for (p, xStart, yStart, xEnd, yEnd) in bound_boxes:
            xmax = max(valid_boxes[0][0], xStart)
            ymax = max(valid_boxes[0][1], yStart)
            xmin = min(valid_boxes[0][2], xEnd)
            ymin = min(valid_boxes[0][3], yEnd)
            
            # compute the area of intersection rectangle
            interArea = (xmin - xmax + 1) * (ymin - ymax + 1)

            box1 = (valid_boxes[0][2] - valid_boxes[0][0] + 1) * (valid_boxes[0][3] - valid_boxes[0][1] + 1)
            box2 = (xEnd - xStart + 1) * (yEnd - yStart + 1)
            # compute the intersection over union by taking the intersection
            # area and dividing it by the sum of prediction + ground-truth
            # areas - the interesection area
            iou = interArea / float((box1 + box2) - interArea)
            if interArea < 0:
                if abs(iou) >= overlap_threshold:
                    # add to remove list and increase index by 1
                    bound_boxes.pop(index)
                    index = index+1
                else:
                    index = index+1
            else:
                continue
        
    return (valid_boxes)

# Main

In [16]:
# Import the test images
training_images = training_images_to_dictionary("dataset/Train/") 

# Initilise SIFT
sift = init_sift()

# Create a dictionary with of sift descriptors for each training image
feature_dictionary_training = extract_features_and_order(training_images)

# Create a list of the features relating to the training images
feature_list = extract_features(training_images)

# cluster features to create visual words
visual_words = cluster_algorithm(feature_list, 50)

# Creates histograms for train data    
bovw_train = image_class(feature_dictionary_training, visual_words) 

# Stack feature histograms
Data1 = np.asarray(bovw_train['car'], dtype=np.float32)
Data2 = np.asarray(bovw_train['not_car'], dtype=np.float32)
Data = np.vstack((Data1, Data2))

# Create labels
label1 = np.ones(len(bovw_train['car']), dtype=int)
label2 = np.negative(np.ones(len(bovw_train['not_car']), dtype=int))
labels = np.append(label1, label2)

In [17]:
# Define SVM
my_SVM = SVM_parameters(0.0001, 100, True)

# Train the SVM
SVM_train(my_SVM,Data,labels)

In [19]:
# Define parameters for windowing operation
# Define windowsize
(winW, winH) = (72,180)
# Define stepSize
stepSize = 15
# Define images
image = cv2.imread("dataset/TestImage/Carpark_1.jpg", 0)
image = cv2.resize(image, (1920,1080))
image_colour = cv2.imread("dataset/TestImage/Carpark_1.jpg")
image_colour = cv2.resize(image_colour, (1920,1080))
image_colour = image_colour.copy()
box_details = []
# slide window across image
for (x, y, window) in sliding_window(image, stepSize, windowSize=(winW, winH)):
    if (window.shape[0] != winH) or (window.shape[1] != winW):
        continue

     # get features (descriptors) of the current window
    window_descriptors = create_features(window, sift)

    if window_descriptors is not None:

        #list for combining bouding box coordinates and it probability
        probability_and_location = []

        # create a histogram of those features
        BagOfFeatures_window = histogram(window_descriptors, visual_words)

        # find probabilty that the window belongs to class 'car'
        probability = (my_SVM.predict_proba([BagOfFeatures_window]))[:,1]
        # represent percentage as a whole nubmer (divide by 100 to get perecentage to 2 decimal places)
        probability = probability*10000
        # if the probability is greater than or equal to 80 percent then draw a bounding box
        if probability >= 8000:
            # put bounding box coordinates and probability in an array
            probability_and_location = [probability[0], x, y, (x+winW), (y+winH)]
            box_details.append(np.array(probability_and_location))
            # draw bounding box and display image
            cv2.rectangle(image_colour, (x, y), (x + winW, y + winH), (0, 255, 0),2)
            cv2.namedWindow('Carpark',cv2.WINDOW_NORMAL)
            cv2.imshow("Carpark", image_colour)   
            cv2.waitKey(1)
            time.sleep(0.025)

        else:
            continue
else:
    # Save image and close window
    cv2.imwrite("Detection_without_NMS.jpg", image_colour)
    cv2.destroyAllWindows()  
    print("finished")

finished


In [30]:
image_colour = cv2.imread("dataset/TestImage/Carpark_1.jpg")
image_colour = cv2.resize(image_colour, (1920,1080))
image_colour = image_colour.copy()

pick = nms(box_details, 0.5)
for (p, startX, startY, endX, endY) in pick:
    cv2.rectangle(image_colour, (startX, startY), (endX, endY), (0, 255, 0), 2)
    cv2.namedWindow('After NMS',cv2.WINDOW_NORMAL)
    cv2.imshow("After NMS", image_colour)
    cv2.waitKey(1)
    time.sleep(0.025)
else:
    cv2.imwrite("After NMS.jpg", image_colour)
    cv2.destroyAllWindows()  
    print("finished")

finished
